# Visualisering av slektstre

I denne notebooken utforsker vi alle tilgjengelige visualiseringsalternativer for slektstre.

## Tilgjengelige visualiseringer

1. **Hierarkisk tre** - Tradisjonell struktur
2. **Fan chart** - Sirkulær visning
3. **Interaktiv tre** - Plotly-basert
4. **Hourglass view** - Fokusperson i midten
5. **Statistikk** - Grafer og diagrammer


In [ ]:
# Importer nødvendige biblioteker
import sys
sys.path.append('../src')

from models import Person, Ekteskap, FamilieData, Gender
from tree import Slektstre
from family_io import load_from_yaml
from visualization import (
    plot_hierarchical_tree,
    plot_fan_chart,
    plot_interactive_tree,
    plot_statistics,
    plot_hourglass_view
)
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import date

print("✅ Alle biblioteker importert!")


In [ ]:
# Last eksempel-familie
familie_data = load_from_yaml('../data/eksempel_familie.yaml')
slektstre = Slektstre(familie_data)

print(f"Familie lastet med {len(familie_data.personer)} personer")
print(f"Beskrivelse: {familie_data.beskrivelse}")

# Vis statistikk
stats = slektstre.get_statistics()
print(f"\n📊 Statistikk:")
print(f"Antall generasjoner: {stats['max_generation'] + 1}")
print(f"Gjennomsnittsalder: {stats['average_age']:.1f} år")
print(f"Antall ekteskap: {stats['total_marriages']}")


## 1. Hierarkisk tre

Tradisjonell struktur med generasjoner fra topp til bunn:


In [ ]:
# Plott hierarkisk tre
fig = plot_hierarchical_tree(slektstre, title="Hierarkisk slektstre")
plt.show()


## 2. Fan Chart

Sirkulær visning med eldste generasjon i midten:


In [ ]:
# Plott fan chart
fig = plot_fan_chart(slektstre, title="Fan Chart - Sirkulær visning")
plt.show()


## 3. Interaktiv tre

Plotly-basert interaktiv visualisering med hover-info:


In [ ]:
# Plott interaktiv tre
fig = plot_interactive_tree(slektstre, title="Interaktiv slektstre")
fig.show()


## 4. Hourglass View

Fokusperson i midten med forfedre over og etterkommere under:


In [ ]:
# Velg en fokusperson (Arvid Lundervold)
fokusperson_id = "p3"  # Arvid Lundervold
fokusperson = slektstre.get_person(fokusperson_id)

if fokusperson:
    print(f"Fokusperson: {fokusperson.fullt_navn} (ID: {fokusperson.id})")
    
    # Plott hourglass view
    fig = plot_hourglass_view(slektstre, fokusperson_id, title=f"Hourglass View - {fokusperson.fullt_navn}")
    plt.show()
else:
    print("Fokusperson ikke funnet!")


## 5. Statistikk og diagrammer

Vis ulike statistikk-diagrammer:


In [ ]:
# Plott statistikk
fig = plot_statistics(slektstre, title="Familie-statistikk")
plt.show()


## 6. Tilpassede visualiseringer

La oss eksperimentere med forskjellige parametere:


In [ ]:
# Hierarkisk tre med større figurstørrelse
fig = plot_hierarchical_tree(
    slektstre, 
    title="Stort hierarkisk tre",
    figsize=(20, 12)
)
plt.show()


In [ ]:
# Fan chart med større radius
fig = plot_fan_chart(
    slektstre, 
    title="Stor fan chart",
    figsize=(15, 15),
    radius=8
)
plt.show()


## 7. Sammenligning av visualiseringer

La oss sammenligne forskjellige visualiseringer side ved side:


In [ ]:
# Sammenlign alle visualiseringer
fig, axes = plt.subplots(2, 2, figsize=(20, 16))

# Hierarkisk tre
plot_hierarchical_tree(slektstre, ax=axes[0,0], title="Hierarkisk")
axes[0,0].set_title("Hierarkisk tre", fontsize=14, fontweight='bold')

# Fan chart
plot_fan_chart(slektstre, ax=axes[0,1], title="Fan Chart")
axes[0,1].set_title("Fan Chart", fontsize=14, fontweight='bold')

# Hourglass view
plot_hourglass_view(slektstre, "p3", ax=axes[1,0], title="Hourglass")
axes[1,0].set_title("Hourglass View", fontsize=14, fontweight='bold')

# Statistikk
plot_statistics(slektstre, ax=axes[1,1], title="Statistikk")
axes[1,1].set_title("Statistikk", fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()


## 8. Eksport av visualiseringer

La oss lagre visualiseringer til filer:


In [ ]:
# Lagre visualiseringer til filer
import os

# Opprett mappe for eksporterte bilder
os.makedirs("eksporterte_bilder", exist_ok=True)

# Lagre hierarkisk tre
fig = plot_hierarchical_tree(slektstre, title="Hierarkisk slektstre")
fig.savefig("eksporterte_bilder/hierarkisk_tre.png", dpi=300, bbox_inches='tight')
plt.close(fig)

# Lagre fan chart
fig = plot_fan_chart(slektstre, title="Fan Chart")
fig.savefig("eksporterte_bilder/fan_chart.png", dpi=300, bbox_inches='tight')
plt.close(fig)

# Lagre hourglass view
fig = plot_hourglass_view(slektstre, "p3", title="Hourglass View")
fig.savefig("eksporterte_bilder/hourglass_view.png", dpi=300, bbox_inches='tight')
plt.close(fig)

# Lagre statistikk
fig = plot_statistics(slektstre, title="Statistikk")
fig.savefig("eksporterte_bilder/statistikk.png", dpi=300, bbox_inches='tight')
plt.close(fig)

print("✅ Alle visualiseringer lagret til 'eksporterte_bilder/' mappen")
print("📁 Filene:")
for fil in os.listdir("eksporterte_bilder"):
    print(f"  - {fil}")


## 9. Rydde opp

La oss slette de eksporterte bildene:


In [ ]:
# Slett eksporterte bilder
import shutil

if os.path.exists("eksporterte_bilder"):
    shutil.rmtree("eksporterte_bilder")
    print("🗑️ Slettet 'eksporterte_bilder/' mappen")

print("✅ Opprydding fullført!")


## Oppsummering

I denne notebooken har du utforsket alle visualiseringsalternativer:

1. ✅ **Hierarkisk tre** - Tradisjonell struktur, god for oversikt
2. ✅ **Fan chart** - Sirkulær visning, visuelt tiltalende
3. ✅ **Interaktiv tre** - Plotly-basert, zoom og hover-info
4. ✅ **Hourglass view** - Fokusperson i midten, god for detaljer
5. ✅ **Statistikk** - Grafer og diagrammer, dataanalyse
6. ✅ **Tilpassede visualiseringer** - Forskjellige størrelser og parametere
7. ✅ **Sammenligning** - Side ved side visning
8. ✅ **Eksport** - Lagre til PNG-filer
9. ✅ **Opprydding** - Slette midlertidige filer

**Anbefalinger**:
- Bruk **hierarkisk tre** for generell oversikt
- Bruk **fan chart** for presentasjoner
- Bruk **interaktiv tre** for utforskning
- Bruk **hourglass view** for fokus på en person
- Bruk **statistikk** for dataanalyse

**Neste steg**: Du har nå fullført alle notebookene! Du kan begynne å bygge ditt eget slektstre.
